In [ ]:
import numpy as np
import pandas as pd
import altair as alt
from altair import datum, expr
alt.data_transformers.enable('json')
alt.data_transformers.disable_max_rows()
alt.renderers.enable('notebook')

filenames_and_names_tuples = [('benchmark_cbsh_no_lpa_timeout_3600.csv', 'CBS',
                               'benchmark_idcbsh_lpa_with_lpmdd_and_path_repair_timeout_3600-in-container.csv', 'IDCBS',
                               True, 3600, ),
                              ('benchmark_ecbsh_1_01_no_lpa.csv', 'ECBS(1.01)', 
                               'benchmark_ecbsh_1_01_lpa_up_and_down_5_child_pref_budget_20_pref_options_with_lpmdd_and_path_repair.csv', 'ECBS(1.01) w/ LCA-jumping',
                               'benchmark_ecbsh_1_05_no_lpa.csv', 'ECBS(1.05)', 
                               'benchmark_ecbsh_1_05_lpa_up_and_down_5_child_pref_budget_20_pref_options_with_lpmdd_and_path_repair.csv', 'ECBS(1.05) w/ LCA-jumping', 
#                                'benchmark_cbsh_no_lpa.csv', 'CBS',
#                                'benchmark_ecbsh_1_00_lpa_up_and_down_5_child_pref_budget_20_pref_options_with_lpmdd_and_path_repair.csv', 'CBS w/ LCA-jumping', 
                               True, 60),
                             ]
filenames_and_names_tuple = filenames_and_names_tuples[0]

only_first_scenario = filenames_and_names_tuple[-2]
data_parts = []
solver_names = []
for i in range(int(len(filenames_and_names_tuple) / 2 - 1)):
    data_part = pd.read_csv(filenames_and_names_tuple[2*i],
                  na_values=['=NA()'],  # Additional #NA values
                 )
    data_part.solver = filenames_and_names_tuple[2*i + 1]
    solver_names.append(filenames_and_names_tuple[2*i + 1])
    data_parts.append(data_part)
    
# cbs = pd.read_csv(filenames_and_names_tuple[0],
#                   na_values=['=NA()'],  # Additional #NA values
#                  )
# baseline_name = filenames_and_names_tuple[2]
# cbs.solver = baseline_name
# idcbs_lpa = pd.read_csv(filenames_and_names_tuple[1], na_values=['=NA()'])
# improved_name = filenames_and_names_tuple[3]
# idcbs_lpa.solver = improved_name
# data = pd.concat([cbs, idcbs_lpa])
data = pd.concat(data_parts)
data.Runtime.fillna(data['Wall Runtime'], inplace=True)  # When memory runs out we only have the wall runtime (timed externally by the script)
#data['Max Mem (kB)'].replace(to_replace={'8g': 8000000}, inplace=True)
data.drop(
    columns=set(data.columns) - {'Runtime', 'instance', 'solver', 'HL Expanded', 'HL Generated',
                                 'Num of Agents', 'MDD Time', 'HL Heuristic Time', 'Up&Down Time',
                                 'HL runtime', 'LL runtime', 'CAT Time', 'HL Node Verification Time',
                                 'Max Mem (kB)', 'Cost'
                                }, inplace=True)  # Drop columns we aren't using to speed things up later
data['timeout'] = filenames_and_names_tuple[-1]
data['mem_usage_cap'] = 8000  # In mB
map_open_cells_counts = { # From the benchmarks page
    'ost003d':13214,
    'den502d':27235,
    'den520d':28178,
    'brc202d':43151,
    'empty-8-8':64,
    'empty-16-16':256,
    'empty-32-32':1024,
    'empty-48-48':2304,
    'random-32-32-10':922,
    'random-32-32-20':819,
    'random-64-64-10':3687,
    'random-64-64-20':3270,
    'maze-128-128-1':8191,
    'maze-128-128-2':10858,
    'maze-128-128-10':14818,
    'maze-32-32-2':666,
    'room-32-32-4':682,
    'room-64-64-8':3232,
    'room-64-64-16':3648,
    'den312d':2445,
    'orz900d':96603,
    'ht_chantry':7461,
    'ht_mansion_n':8959,
    'lak303d':14784,
    'lt_gallowstemplar_n':10021,
    'w_woundedcoast':34020,
    'Berlin_1_256':47540,
    'Boston_0_256':47768,
    'Paris_1_256':47240,
    'warehouse-10-20-10-2-1':5669,
    'warehouse-10-20-10-2-2':9776,
    'warehouse-20-40-10-2-1':22599,
    'warehouse-20-40-10-2-2':38756,
}
map_groups = {
    'ost003d':'Games',
    'den502d':'Games',
    'den520d':'Games',
    'brc202d':'Games',
    'empty-8-8':'Empty',
    'empty-16-16':'Empty',
    'empty-32-32':'Empty',
    'empty-48-48':'Empty',
    'random-32-32-10':'Random',
    'random-32-32-20':'Random',
    'random-64-64-10':'Random',
    'random-64-64-20':'Random',
    'maze-128-128-1':'Mazes',
    'maze-128-128-2':'Mazes',
    'maze-128-128-10':'Mazes',
    'maze-32-32-2':'Mazes',
    'room-32-32-4':'Rooms',
    'room-64-64-8':'Rooms',
    'room-64-64-16':'Rooms',
    'den312d':'Games',
    'orz900d':'Games',
    'ht_chantry':'Games',
    'ht_mansion_n':'Games',
    'lak303d':'Games',
    'lt_gallowstemplar_n':'Games',
    'w_woundedcoast':'Games',
    'Berlin_1_256':'City',
    'Boston_0_256':'City',
    'Paris_1_256':'City',
    'warehouse-10-20-10-2-1':'Warehouse',
    'warehouse-10-20-10-2-2':'Warehouse',
    'warehouse-20-40-10-2-1':'Warehouse',
    'warehouse-20-40-10-2-2':'Warehouse',
}
data['open_map_cells'] = data['instance'].apply(lambda x: x.rsplit('-', 2)[0]).str.partition('scen/')[2].map(map_open_cells_counts)
data['map_group'] = data['instance'].apply(lambda x: x.rsplit('-', 2)[0]).str.partition('scen/')[2].map(map_groups)
data['mem_usage_per_second'] = data['Max Mem (kB)'] / data['Runtime']
data.rename(columns={'HL Expanded': 'HL_Expanded'}, inplace=True)
data.rename(columns={'HL Generated': 'HL_Generated'}, inplace=True)
data.rename(columns={'Max Mem (kB)': 'Max_Mem'}, inplace=True)
data.Max_Mem /= 1000_000  # Gigabytes (https://en.wikipedia.org/wiki/Mebibyte)

instance_dropdown = alt.binding_select(options=[None] + list(data.instance.unique()), )
instance_selection = alt.selection_single(fields=['instance'], bind=instance_dropdown, name='Selected',
#                                           empty='none',  # When enabled the chart is drawn in size 0x0 and isn't updated later
                                         )

runtime_nearest_x = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Num of Agents'], empty='none')


runtime_chart = alt.Chart().mark_line(strokeWidth=4)\
    .encode(alt.X('Num of Agents',
                  type='ordinal',
                  scale=alt.Scale(
#                       domain=(2,XXX),  Set Values here
                  ),
                  axis=alt.Axis(titleFontSize=14, 
                                #labelFontWeight=alt.FontWeight(500)
                                #labelFontStyle=alt.FontStyle('bold')
                                labelFontSize=14, 
                               )  # labelFontWeight=400 is normal, 700 is bold, multiples of 100 only
                 ),
            alt.Y('Runtime',
                  type='quantitative',
                  scale=alt.Scale(
#                       domain=(0,3650), # Doesn't work well with log scale
                      clamp=True,
                      type='log',
                  ),
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='Runtime (s)', titleFontSize=14,
                                labelFontSize=14
                               ),  # Force the title not to include ",timeout" from the other layer
                  #impute=alt.ImputeParams(value=36000),
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(#title="Solver",
                                        title='',
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=200,
                                        symbolStrokeWidth=3, labelLimit=600, orient='bottom',
                                        offset=0,)))
#    .transform_filter(datum.Runtime<3600)\
#     .transform_filter(datum.Runtime>=1)  Doesn't work well with impute if one of the solvers is above and the other isn't



runtime_timeout_rule = alt.Chart()\
    .mark_rule(color='firebrick')\
    .encode(
        y=alt.Y(
            'timeout',
        ),
        size=alt.SizeValue(2)
    )

# runtime_timeout_text = alt.Chart()\
#     .mark_text(
#         align='right',
#         baseline='middle',
#         dx=0,
#         dy=-5,  # Slightly above the line
#         size=8,
#         fontWeight=200,  # Half of weight of fontStyle='normal'
#     )\
#     .encode(
#         text=alt.value('timeout'),
#         color=alt.value('black')
#     )

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor when it isn't immediately on a line.
runtime_selectors = alt.Chart().mark_point().encode(
    x='Num of Agents:O',
    opacity=alt.value(0),
).add_selection(  # Make the selection select 
    runtime_nearest_x
)

runtime_nearest_points = runtime_chart.mark_point()\
    .encode(opacity=alt.condition(runtime_nearest_x, alt.value(1), alt.value(0)),)\
    .transform_filter(runtime_nearest_x)

runtime_nearest_text = runtime_nearest_points.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(runtime_nearest_x, 'Runtime:Q', alt.value(''))
)

runtime_nearest_rule = runtime_nearest_text.mark_rule(color='gray')\
    .encode(x='Num of Agents:O')\
    .transform_filter(runtime_nearest_x)\
    .add_selection(instance_selection)  # Add the selection on the latest mark of the data (it may only be added once)

runtime_chart = alt.layer(runtime_chart,
                          runtime_timeout_rule,
#                           runtime_timeout_text,
                          runtime_selectors,
                          runtime_nearest_points,
                          runtime_nearest_text,
                          runtime_nearest_rule,
                          data=data)\
    .transform_filter(instance_selection)\
    .properties(
        width=400,
        height=250
    )

mem_usage_chart = alt.Chart()\
    .mark_line(strokeWidth=4)\
    .encode(alt.X('Num of Agents', type='ordinal',
                  axis=alt.Axis(titleFontSize=14, 
                                labelFontSize=14,
                               )
                 ),
            alt.Y('Max_Mem',
                  type='quantitative',
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='Memory Usage (GB)',
                                titleFontSize=14, 
                                labelFontSize=14, 
                               ),
                  scale=alt.Scale(type='log',)
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(#title="Solver",
                                        title='',
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=200,
                                        symbolStrokeWidth=3, labelLimit=600, orient='bottom',
                                        offset=0,)))\
    .transform_filter(instance_selection)\
    .properties(
        width=400,
        height=250
    )

mem_usage_cap_rule = alt.Chart()\
    .mark_rule(color='firebrick')\
    .encode(
        y=alt.Y(
            'mem_usage_cap',
            type='quantitative',
        ),
        size=alt.SizeValue(2)
    )

mem_usage_chart = alt.layer(mem_usage_chart,
                            mem_usage_cap_rule,
                            data=data
                           )

hl_expanded_chart = alt.Chart(data)\
    .mark_line(strokeWidth=4)\
    .encode(alt.X('Num of Agents', type='ordinal',
                  axis=alt.Axis(titleFontSize=14, 
                                labelFontSize=14, 
                               )
                 ),
            alt.Y('HL_Expanded',
                  type='quantitative',
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='HL Expanded Nodes', titleFontSize=14,
                                labelFontSize=14,
                               ),
                  #scale=alt.Scale(type='log',
                                 #domain=(0, 1000), clamp=True,
                  #               ),
                  #impute=alt.ImputeParams(value=10000),  # There isn't a way in vegalite 3, which altair is based on, to impute with a non-preset function
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(#title='Solver'
                                        title='', titleFontSize=16,
                                        labelFontSize=16, symbolSize=200,
                                        symbolStrokeWidth=3, labelLimit=600, orient='bottom',
                                        offset=0,)))\
    .transform_filter(instance_selection)\
    .properties(
        width=400,
        height=250
    )
#    .transform_filter(datum.Runtime<60)\

# hl_expanded_points = hl_expanded_chart.mark_point()\
#     .encode(opacity=alt.condition(nearest, alt.value(1), alt.value(0)))\
#     .add_selection(nearest)
# hl_expanded_chart = alt.layer(hl_expanded_chart, hl_expanded_points)

hl_generated_chart = alt.Chart(data)\
    .mark_line(strokeWidth=4)\
    .encode(alt.X('Num of Agents', type='ordinal',
                  axis=alt.Axis(titleFontSize=14, 
                                labelFontSize=14, 
                               )
                 ),
            alt.Y('HL_Generated',
                  type='quantitative',
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='HL Generated Nodes', titleFontSize=14,
                                labelFontSize=14,
                               ),
                  scale=alt.Scale(type='log',
                                 #domain=(0, 1000), clamp=True,
                                 ),
                  #impute=alt.ImputeParams(value=10000),
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(title='', labelFontSize=16, symbolSize=200,
                                        symbolStrokeWidth=3, labelLimit=600, orient='bottom',
                                        offset=0,)))\
    .transform_filter(instance_selection)\
    .properties(
        width=400,
        height=250
    )
#    .transform_filter(datum.Runtime<60)\
#    .interactive()  not good

# TODO: Add selectors and rules for the bottom two charts

joint_chart = alt.vconcat(runtime_chart, mem_usage_chart,
#                           hl_expanded_chart,
                          hl_generated_chart)
joint_chart

In [ ]:
all_success_row_groups = []
success_rate_rows = []
from collections import Counter
successfully_solved = Counter()
num_problem_instances = num_all_failures = num_actual_instances = 0
max_num_agents = data['Num of Agents'].max()
solver_names_set = set(solver_names)
for instance, instance_rows in data.groupby(['instance',]):
    if only_first_scenario and not instance.endswith('-1.scen'):
        print(f'Skipping scenario {instance}')
        continue
    print(f'scenario {instance}')
    map_size = instance_rows['open_map_cells'].values[0]
    map_group = instance_rows['map_group'].values[0]
    for num_agents in range(2, max_num_agents + 1):
        num_problem_instances += 1
        instance_num_agents_rows = instance_rows.loc[instance_rows['Num of Agents'] == num_agents]
        if all(instance_num_agents_rows.Cost.values < 0): # Also works if there are zero rows
            num_all_failures += 1
        if len(instance_num_agents_rows) != 0:
            num_actual_instances += 1
        instance_num_agents_success_rows = instance_num_agents_rows.loc[instance_num_agents_rows.Cost >= 0]
        instance_num_agents_explicit_failures_rows = instance_num_agents_rows.loc[instance_num_agents_rows.Cost < 0]
        if len(instance_num_agents_success_rows) == len(solver_names):
            all_success_row_groups.append(instance_num_agents_success_rows)
        for solver_name in list(instance_num_agents_success_rows.solver.values):
            success_rate_rows.append((num_agents, instance, 1, map_size, solver_name, map_group, None))
            successfully_solved[solver_name] += 1
        for index, row in instance_num_agents_explicit_failures_rows.iterrows():
            success_rate_rows.append((num_agents, instance, 0, map_size, row.solver, map_group, {-2:'Memory failure', -1:'Timeout failure'}[row.Cost]))
        for solver_name in solver_names_set - set(instance_num_agents_success_rows.solver.values) - set(instance_num_agents_explicit_failures_rows):
            success_rate_rows.append((num_agents, instance, 0, map_size, solver_name, map_group, 'Implicit failure'))
all_success_data = pd.concat(all_success_row_groups)
success_rate_data = pd.DataFrame(success_rate_rows, columns=('Num of Agents', 'instance', 'success', 'map_size', 'solver', 'map_group', 'failure_reason'))
num_all_success_groups = len(all_success_row_groups)
del all_success_row_groups
del success_rate_rows
print(f'num_problem_instances={num_problem_instances}, '
      f'all solved: {num_all_success_groups}, '
      f'none solved:{num_all_failures}, '
      f'actual instances: {num_actual_instances}'
)
for solver_name in solver_names:
    print(f'{solver_name} solved: {successfully_solved[solver_name]}')

In [ ]:
success_rate_order = {
    'ECBS(1.05) w/ LCA-jumping': 1,
    'ECBS(1.01) w/ LCA-jumping': 2,
    'ECBS(1.05)': 3,
    'ECBS(1.01)': 4,
    'CBS w/ LCA-jumping': 5,
    'CBS': 6,
    
    'IDCBS': 101,
    'CBS': 102,
}
success_rate_data['success_rate_order'] = success_rate_data['solver'].map(success_rate_order)

success_rate_by_agents = alt.Chart(success_rate_data)\
    .mark_line(strokeWidth=5)\
    .encode(alt.X('Num of Agents', type='quantitative',
#                   bin=alt.BinParams(step=5),
                  axis=alt.Axis(titleFontSize=16, 
                                labelFontSize=16,
#                                 title='Num of Agents (5-agent bins)',
                                title='Num of Agents',
                                titlePadding=-3,
                               ),
#                   scale=alt.Scale(type='log')
                 ),
            alt.Y('success', type='quantitative', aggregate='mean',
                  #domain=(0, 3650), clamp=True),
                  axis=alt.Axis(orient=alt.AxisOrient('left'),
                                title='Success Rate',
                                titleFontSize=16,
                                labelFontSize=16,
                               )
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(title='',
                                        #title="Solver:",
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=200, symbolStrokeWidth=3,
                                        labelLimit=600, orient='right',
                                        offset=0, #symbolType='circle',
                                        #symbolFillColor=,
                                       ),
                      sort=alt.Sort(field='success_rate_order', op='mean'),
                     ),
    )\
    .properties(
            width=750,
            height=180,
    )

success_rate_by_map_size = alt.Chart(success_rate_data)\
    .mark_line(strokeWidth=2.5)\
    .encode(alt.X('map_size', type='quantitative', bin=alt.BinParams(step=1500),
                  axis=alt.Axis(titleFontSize=14, 
                                labelFontSize=14,
                                title='#Open (Unblocked) Map Cells',
                                titlePadding=0,
                               ),
#                   scale=alt.Scale(type='log')
                 ),
            alt.Y('success', type='quantitative', aggregate='mean',
                  #domain=(0, 3650), clamp=True),
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='Success Rate', titleFontSize=14,
                                labelFontSize=14,
                               )
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(title='',
                                        #title="Solver:",
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=200, symbolStrokeWidth=3,
                                        labelLimit=600, orient='bottom',
                                        offset=0,
                                       ),
                       sort=alt.Order(field='success_rate_order', op='mean')),
            
    )\
    .properties(
            width=500,
            height=750,
    )

# success_rate = alt.vconcat(success_rate_by_agents, success_rate_by_map_size)
# success_rate_by_map_size
success_rate_by_agents
# success_rate

In [ ]:
rates = []
print('group,', end='')
for solver_name in solver_names:
    print(f'{solver_name} solved,', end='')
for solver_name in solver_names:
    print(f'{solver_name} timeouts,', end='')
for solver_name in solver_names:
    print(f'{solver_name} memory failures,', end='')
for solver_name in solver_names:
    print(f'{solver_name} implicit failures,', end='')
print()
for group, rows in success_rate_data.groupby(['map_group']):
    print(f'{group},', end='')
    for solver_name in solver_names:
        print(f'{len(rows.loc[(rows.solver == solver_name) & (rows.success > 0)])},', end='')
    for solver_name in solver_names:
        print(f'{len(rows.loc[(rows.solver == solver_name) & (rows.failure_reason == "Timeout failure")])},', end='')
    for solver_name in solver_names:
        print(f'{len(rows.loc[(rows.solver == solver_name) & (rows.failure_reason == "Memory failure")])},', end='')
    for solver_name in solver_names:
        print(f'{len(rows.loc[(rows.solver == solver_name) & (rows.failure_reason == "Implicit failure")])},', end='')
    print()
    rates.append(alt.Chart(rows)\
    .mark_line(strokeWidth=4)\
    .encode(alt.X('Num of Agents', type='quantitative', bin=alt.BinParams(step=5),
                  axis=alt.Axis(titleFontSize=14, 
                                labelFontSize=14,
                                title='Num of Agents',
                                #titlePadding=0,  # Works but not pretty
                               ),
                 ),
            alt.Y('success', type='quantitative', aggregate='mean',
                  #domain=(0, 3650), clamp=True),
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='Success Rate', titleFontSize=14,
                                labelFontSize=14,
                               )
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(title='',
                                        #title="Solver:",
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=200, symbolStrokeWidth=3,
                                        labelLimit=600, orient='bottom',
                                        offset=0, #symbolType='circle',
                                        #symbolFillColor=,
                                       ),
                       sort=alt.Sort(field='success_rate_order', op='mean')),
    )\
    .properties(
            width=750,
            height=200,
            title=f'{group}'
    ))
    
alt.vconcat(*rates)

In [ ]:
# All data:
average_runtime = alt.Chart(all_success_data)\
    .mark_line(strokeWidth=4)\
    .encode(alt.X('Num of Agents', type='ordinal',
                  axis=alt.Axis(titleFontSize=16, 
                                labelFontSize=16,
                                titlePadding=-3,
                               ),
                 ),
            alt.Y('Runtime', type='quantitative', aggregate='mean',
                  scale=alt.Scale(type='log',
#                                   domain=(0, 1800), clamp=True,
                  ),
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='Average Runtime (s)',
                                titleFontSize=16,
                                labelFontSize=16,
                               )
                  #impute=alt.ImputeParams(value=36000),
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(title='',
                                        #title="Solver:",
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=500, symbolStrokeWidth=3,
                                        labelLimit=600, orient='right',
#                                         offset=0,
                                       )))\
    .properties(
            width=400,
            height=250
    )

# runtime_timeout_rule = alt.Chart(all_success_data)\
#     .mark_rule(color='firebrick')\
#     .encode(
#         y=alt.Y(
#             'timeout',
#         ),
#         size=alt.SizeValue(2)
#     )

# average_runtime = alt.layer(average_runtime, runtime_timeout_rule)\
#     .properties(
#             width=400,
#             height=250
#     )

average_mem_usage = alt.Chart(all_success_data)\
    .mark_line(strokeWidth=4)\
    .encode(alt.X('Num of Agents', type='ordinal',
                  axis=alt.Axis(titleFontSize=14, 
                                labelFontSize=14, 
                               ),
                 ),
            alt.Y('Max_Mem', type='quantitative', aggregate='median',
#                   scale=alt.Scale(type='log',),
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='Median Memory Usage (GB)', titleFontSize=14,
                                labelFontSize=14,
                               ),
                  #impute=alt.ImputeParams(value=36000),
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(#title="Solver:",
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=500, symbolStrokeWidth=3,
                                        labelLimit=600, orient='bottom', offset=0,)))\
    .properties(
            width=400,
            height=250
    )

average_expanded = alt.Chart(all_success_data)\
    .mark_line(strokeWidth=4)\
    .encode(alt.X('Num of Agents', type='ordinal',
                  axis=alt.Axis(titleFontSize=14, 
                                labelFontSize=14, 
                               ),
                 ),
            alt.Y('HL_Expanded', type='quantitative', aggregate='median',
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='Median Expanded Nodes', titleFontSize=14,
                                labelFontSize=14,
                               ),
#                   scale=alt.Scale(type='log',),
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(#title="Solver:",
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=500, symbolStrokeWidth=3,
                                        labelLimit=600, orient='bottom', offset=0)))\
    .properties(
            width=400,
            height=250
    )

average_generated = alt.Chart(all_success_data)\
    .mark_line(strokeWidth=2.5)\
    .encode(alt.X('Num of Agents', type='ordinal',
                  axis=alt.Axis(titleFontSize=14, 
                                labelFontSize=14,
                                titlePadding=-3,
                               ),
                 ),
            alt.Y('HL_Generated', type='quantitative', aggregate='mean',
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='Average Generated Nodes', titleFontSize=14,
                                labelFontSize=14,
                               ),
                  scale=alt.Scale(type='log',),
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(title='',
                                        #title="Solver:",
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=500, symbolStrokeWidth=3,
                                        labelLimit=600, orient='bottom', offset=0, )))\
    .properties(
            width=400,
            height=250
    )

alt.vconcat(average_runtime,
#             average_mem_usage,
#             average_expanded,
#             average_generated,
           )

In [ ]:
print(f'#agents,#instances,', end='')
for solver_name in solver_names:
    print(f'{solver_name} average #generated,', end='')
print()
for num_agents, rows in all_success_data.groupby(['Num of Agents', ]):
    print(f'{num_agents},{int(len(rows) / len(solver_names))},', end='')
    for solver_name in solver_names:
        print(f"{np.average(rows.loc[rows.solver==solver_name]['HL_Generated'])},", end='')
    print()
#     print(rows)
#     break


In [ ]:
#data['log_hl_generated'] = np.log(data['HL_Generated'])
average_mem_usage_by_generated = alt.Chart(data)\
    .mark_line(strokeWidth=2.5)\
    .encode(alt.X('HL_Generated', bin=alt.BinParams(step=100000),
#                   'log_hl_generated', bin=alt.BinParams(step=0.100000),
                  type='quantitative', 
#                   scale=alt.Scale(type='log'),
                  axis=alt.Axis(title='HL Generated Nodes', titleFontSize=14, 
                                labelFontSize=14, 
                               ),
                 ),
            alt.Y('Max_Mem', type='quantitative', aggregate='mean',
#                   scale=alt.Scale(type='log',),
                  axis=alt.Axis(orient=alt.AxisOrient('left'), title='Average Memory Usage (GB)', titleFontSize=14,
                                labelFontSize=14,
                               ),
                  #impute=alt.ImputeParams(value=36000),
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(#title="Solver:",
                                        title='',
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=500, symbolStrokeWidth=3,
                                        labelLimit=600, orient='bottom',)))\
    .properties(
            width=400,
            height=250
    )

average_mem_usage_by_time = alt.Chart(data)\
    .mark_line(strokeWidth=5)\
    .encode(alt.X('Runtime', type='quantitative',
                  bin=alt.BinParams(step=120),
                  axis=alt.Axis(title='Final Runtime (120-second bins)',
                                titleFontSize=16, 
                                labelFontSize=16,
                                titlePadding=-2,
                               ),
                 ),
            alt.Y('Max_Mem', type='quantitative', aggregate='mean',
#                   scale=alt.Scale(type='log',),
                  axis=alt.Axis(orient=alt.AxisOrient('left'),
                                title='Average Memory Usage (GB)',
                                titleFontSize=16,
                                labelFontSize=16,
                               ),
                  #impute=alt.ImputeParams(value=36000),
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(#title="Solver:",
                                        title='',
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=500, symbolStrokeWidth=3,
                                        labelLimit=600, orient='right',
#                                         offset=0,
                                       ),
                     ),
           )\
    .properties(
            width=400,
            height=200*22./18*22/21.5,
    )

average_mem_usage_by_map_size = alt.Chart(data)\
    .mark_line(strokeWidth=4)\
    .encode(alt.X('open_map_cells', type='quantitative', #bin=alt.BinParams(step=60),
                  scale=alt.Scale(type='log'), # Doesn't do anything: zero=False, domain=[64,100000], nice=True,
                  axis=alt.Axis(title='#Open (Unblocked) Map Cells', titleFontSize=14, 
                                labelFontSize=14, 
                               ),
                 ),
            alt.Y('Max_Mem',
                #'mem_usage_per_second',
                  type='quantitative', aggregate='mean',
#                   scale=alt.Scale(type='log',),
                  axis=alt.Axis(orient=alt.AxisOrient('left'),
                                title='Average Memory Usage (GB)',
#                                 title='Median Memory Usage Per Second (kB/s)',
                                titleFontSize=14,
                                labelFontSize=14,
                               ),
                  #impute=alt.ImputeParams(value=36000),
                 ),
            alt.Color('solver', type='nominal',
                      legend=alt.Legend(#title="Solver:",
                                        title='',
                                        titleFontSize=16,
                                        labelFontSize=16, symbolSize=500, symbolStrokeWidth=3,
#                                         offset=0,
                                        labelLimit=600, orient='bottom',)))\
    .properties(
            width=400,
            height=250
    )

#alt.vconcat(average_mem_usage_by_generated, average_mem_usage_by_time, average_mem_usage_by_map_size)
average_mem_usage_by_time

In [ ]:
runtime_components_data = data.loc[(data.Cost != -1) & (data.Cost != -2)].melt(
    id_vars=set(data.columns) - {'HL runtime', 'LL runtime',
                                 'Up&Down Time',
                                 'MDD Time', 'HL Heuristic Time',
                                 'CAT Time', 'HL Node Verification Time', #'FAILED',
                                },
    value_vars=['HL runtime', 'LL runtime',
                'Up&Down Time',
                'MDD Time', 'HL Heuristic Time',
                'CAT Time', 'HL Node Verification Time', #'FAILED',
               ],
    var_name='Component',
    value_name='Component_Runtime')


runtime_components_data['Component_Runtime_Per_Generated_Node'] = runtime_components_data['Component_Runtime'] / runtime_components_data['HL_Generated'] * 1000
# runtime_components_data['Component_Runtime_Per_Expanded_Node'] = runtime_components_data['Component_Runtime'] / runtime_components_data['HL_Expanded'] * 1000  # HL_Expanded can be zero - another reason to use HL_Generated

# Better names
runtime_components_data.Component.replace({'HL runtime': 'High-level Work',
                                      'LL runtime': 'Path Finding',
                                      'MDD Time': 'Building MDDs',
                                      'HL Heuristic Time': 'HL Heuristic',
                                      'CAT Time': 'Building CAT',
                                      'HL Node Verification Time': 'Finding Conflicts',
                                      'Up&Down Time': 'LCA-Jumping'
                                      }, inplace=True)

components_instance_dropdown = alt.binding_select(options=[None] + list(runtime_components_data.instance.unique()))
components_instance_selection = alt.selection_single(fields=['instance'],
                                                     bind=components_instance_dropdown, name='Instance',
#                                                      empty='none',  # See top cell for why this isn't enabled
                                                    )

layer_order_top_to_bottom = {
    'Building CAT': 7,  # First because it's shrunk the most
    'LCA-Jumping': 6,
    'Finding Conflicts': 5, # Very thin anyway so order not so important
    'High-level Work': 4,
    'HL Heuristic': 3, # On top of MDD Time because it builds on it
    'Building MDDs': 2,
    'Path Finding': 1, # Last because it's the biggest and shrinks a lot, also, MDD work is on top of it
}
runtime_components_data['layer_order'] = runtime_components_data['Component'].map(layer_order_top_to_bottom)

stacked_runtime_charts = []
for solver_name in solver_names:
    stacked_runtime_charts.append(alt.Chart(runtime_components_data)\
        .mark_area()\
        .encode(alt.X('Num of Agents',
                      type='ordinal',
    #                   scale=alt.Scale(
    #                        domain=list(range(2,79)),  #TODO: Check if this can be set interactively, if yes, try to connect a trigger to update the domain based on that of the other chart
    #                   ),
                      axis=alt.Axis(titleFontSize=14, 
                                    labelFontSize=14, 
                                   ),
                     ),
                alt.Y(
                      #'Component_Runtime',
                      'Component_Runtime_Per_Generated_Node',
                      type='quantitative',
    #                   scale=alt.Scale(domain=(0, 3650), clamp=True,),
                      axis=alt.Axis(orient=alt.AxisOrient('left'),
                                    title='Time Per Generated Node (ms)', titleFontSize=14,
                                    labelFontSize=14,
                                   ),
                     ),
                alt.Color('Component', type='nominal', scale=alt.Scale(
                    range=[
                      "#e41a1c",
    #                   "#4daf4a",
                      "#377eb8",
                      "#984ea3",
                      "#ff7f00",
    #                  "#ffff33",
                      "#f781bf",
                      "#a65628",
                      "#999999",
                    ],
    #                 scheme='category10',
                    #scheme='set1',
                    #scheme='tableau20'
                ),
                legend=alt.Legend(title="Component", titleFontSize=16,
                                  labelFontSize=16, symbolSize=500,
                                  symbolStrokeWidth=3, labelLimit=600, orient='right',
                                  values=list(layer_order_top_to_bottom.keys()),
                                 ),
                ),
                order=alt.Order(field='layer_order', type='ordinal'),
               )\
        .transform_filter(components_instance_selection)\
        .transform_filter(datum.solver == solver_name)\
        .properties(
                width=400,
                height=250,
                title=f'{solver_name} Runtime Components'
        ))
    
    
from functools import reduce
import operator
# joint_chart = reduce(operator.and_, stacked_runtime_charts)\
#     .add_selection(components_instance_selection)\
#     .resolve_scale(x='shared', y='shared', )
joint_chart = reduce(operator.or_, stacked_runtime_charts)\
    .add_selection(components_instance_selection)\
    .resolve_scale(x='shared', y='shared', )

joint_chart

In [ ]:
all_success_runtime_components_data = all_success_data.melt(
    id_vars=set(data.columns) - {'HL runtime', 'LL runtime',
                                 'Up&Down Time',
                                 'MDD Time', 'HL Heuristic Time',
                                 'CAT Time', 'HL Node Verification Time',
                                },
    value_vars=['HL runtime', 'LL runtime',
                'Up&Down Time',
                'MDD Time', 'HL Heuristic Time',
                'CAT Time', 'HL Node Verification Time',
               ],
    var_name='Component',
    value_name='Component_Runtime')


all_success_runtime_components_data['Component_Runtime_Per_Generated_Node'] = all_success_runtime_components_data['Component_Runtime'] / all_success_runtime_components_data['HL_Generated'] * 1000


# Better names
all_success_runtime_components_data.Component.replace({'HL runtime': 'High-level Work',
                                      'LL runtime': 'Path Finding',
                                      'MDD Time': 'Building MDDs',
                                      'HL Heuristic Time': 'HL Heuristic',
                                      'CAT Time': 'Building CAT',
                                      'HL Node Verification Time': 'Finding Conflicts',
                                      'Up&Down Time': 'LCA-Jumping'
                                      }, inplace=True)

layer_order_top_to_bottom = {
    'Building CAT': 7,  # First because it's shrunk the most
    'LCA-Jumping': 6, # Very thin anyway
    'Finding Conflicts': 5, # Very thin anyway so order not so important
    'High-level Work': 4,
    'HL Heuristic': 3, # On top of MDD Time because it builds on it
    'Building MDDs': 2,
    'Path Finding': 1, # Last because it's the biggest and shrinks a lot, also, MDD work is on top of it
}
all_success_runtime_components_data['layer_order'] = all_success_runtime_components_data['Component'].map(layer_order_top_to_bottom)


stacked_runtime_charts = []
for solver_name in solver_names:
    stacked_runtime_charts.append(alt.Chart(all_success_runtime_components_data)\
    .mark_area()\
    .encode(alt.X('Num of Agents',
                  type='quantitative',
#                   bin=alt.BinParams(step=5),
                  scale=alt.Scale(
                      domain=(2,110),  #TODO: Check if this can be set interactively, if yes, try to connect a trigger to update the domain based on that of the other chart
                      clamp=True,
                  ),
                  axis=alt.Axis(titleFontSize=16, 
                                labelFontSize=16,
                                titlePadding=-2,
                               ),
                 ),
            alt.Y(
                  #'Component_Runtime',
                  'Component_Runtime_Per_Generated_Node', aggregate='median',
                  type='quantitative',
                  scale=alt.Scale(domain=(0, 22), clamp=True,),
                  axis=alt.Axis(orient=alt.AxisOrient('left'),
                                title='Mean Time Per Gen. Node (ms)',
                                titleFontSize=16,
                                labelFontSize=16,
                                titlePadding=0,
                               ),
                 ),
            alt.Color('Component', type='nominal', scale=alt.Scale(
                range=[
                  "#e41a1c",
#                   "#4daf4a",
                  "#377eb8",
                  "#984ea3",
                  "#ff7f00",
#                  "#ffff33",
                  "#f781bf",
                  "#a65628",
                  "#999999",
                ],
#                 scheme='category10',
                #scheme='set1',
                #scheme='tableau20'
            ),
                      legend=alt.Legend(title="Component", titleFontSize=16,
                                        labelFontSize=16, symbolSize=500,
                                        symbolStrokeWidth=3, labelLimit=600, orient='right',
                                        values=list(layer_order_top_to_bottom.keys()),
                                       ),
                     ),
            order=alt.Order(field='layer_order', type='ordinal'),
           )\
    .transform_filter(datum.solver == solver_name)\
    .properties(
        width=400,
        height=250,
        title=f'{solver_name} Runtime Breakdown',
    )
   )
    
    

from functools import reduce
import operator
# joint_chart = reduce(operator.and_, stacked_runtime_charts)\
#     .resolve_scale(x='shared', y='shared', )
joint_chart = reduce(operator.or_, stacked_runtime_charts)\
    .resolve_scale(x='shared', y='shared', )\
    .configure_title(fontSize=20)

joint_chart

In [ ]:
data.loc[data['solver']==solver_names[0]]

In [ ]:
data.columns

In [ ]:
runtime_components_data.loc[runtime_components_data['instance'] == '/scen/den520d-random-2.scen']

In [ ]:
runtime_components_data.loc[runtime_components_data.solver==solver_names[1]]

In [ ]:
np.average(
all_success_data.loc[(all_success_data['solver']==solver_names[1]) & (all_success_data['Num of Agents']==70)].HL_Generated)

In [ ]:
data.loc[data.Runtime>3800]

In [ ]:
data